In [69]:
import json

In [70]:
profile_step_to_event_map = {}
# read json file
with open('sample_pytorch_profiler_json_data/withoutfetcher.json') as f:
    data = json.load(f)
    events = data['traceEvents']

    # if user annotation is present, then store it in a dictionary from timestamp to name
    for index,event in enumerate(events):
        if 'cat' in event and event['name'].startswith('Profiler') and not event['name'].startswith('Profiler#0'):
            start = event['ts']
            end = event['ts'] + event['dur']
            profile_step_to_event_map[event['name']] = {}
            for i in range(index+1,len(events)):
                # if start of another event is greater than end of current event, then break
                if events[i]['ts'] > end:
                    break

                # if user annotation is present, then store it in a dictionary from timestamp to name
                if 'cat' in events[i] and 'user_annotation' == events[i]['cat'] and not events[i]['name'].startswith('Optimizer'):
                    event_name = events[i]['name'] if 'DataLoader' not in events[i]['name'] else "DataLoaderWait"
                    profile_step_to_event_map[event['name']][event_name] = events[i]
            # if profile_step_to_event_map[event['name']] is empty dictionary, then remove it
            if not profile_step_to_event_map[event['name']]:
                del profile_step_to_event_map[event['name']]

                    


In [71]:
profile_step_to_event_map

{'ProfilerStep#0': {'DataLoaderWait': {'ph': 'X',
   'cat': 'user_annotation',
   'name': 'enumerate(DataLoader)#_MultiProcessingDataLoaderIter.__next__',
   'pid': 3563708,
   'tid': 3563708,
   'ts': 1695502073087716,
   'dur': 210,
   'args': {'External id': 1974, 'Ev Idx': 949}},
  'move_data_to_device': {'ph': 'X',
   'cat': 'user_annotation',
   'name': 'move_data_to_device',
   'pid': 3563708,
   'tid': 3563708,
   'ts': 1695502073087972,
   'dur': 269,
   'args': {'External id': 1975, 'Ev Idx': 950}},
  'model_forward_pass': {'ph': 'X',
   'cat': 'user_annotation',
   'name': 'model_forward_pass',
   'pid': 3563708,
   'tid': 3563708,
   'ts': 1695502073088253,
   'dur': 5455,
   'args': {'External id': 1984, 'Ev Idx': 959}},
  'model_backward_pass': {'ph': 'X',
   'cat': 'user_annotation',
   'name': 'model_backward_pass',
   'pid': 3563708,
   'tid': 3563708,
   'ts': 1695502073120865,
   'dur': 7702,
   'args': {'External id': 2392, 'Ev Idx': 1367}}},
 'ProfilerStep#1': {'Da

In [72]:
list(list(profile_step_to_event_map.items())[0][1].keys())

['DataLoaderWait',
 'move_data_to_device',
 'model_forward_pass',
 'model_backward_pass']

In [73]:
data_wait_time = []
forward_and_backward_pass_time = []
move_data_to_device_time = []
for step in profile_step_to_event_map:
    data_wait_time.append(profile_step_to_event_map[step]['DataLoaderWait']['dur'])
    forward_pass_start = profile_step_to_event_map[step]['model_forward_pass']['ts']
    backward_pass_end = profile_step_to_event_map[step]['model_backward_pass']['ts'] + profile_step_to_event_map[step]['model_backward_pass']['dur']
    forward_and_backward_pass_time.append(backward_pass_end - forward_pass_start)
    move_data_to_device_time.append(profile_step_to_event_map[step]['move_data_to_device']['dur'])


In [77]:
# print average time for each step
print("Average data wait time: ", sum(data_wait_time)/len(data_wait_time))
print("Average forward + backward pass time: ", sum(forward_and_backward_pass_time)/len(forward_and_backward_pass_time))
print("Average move data to device time: ", sum(move_data_to_device_time)/len(move_data_to_device_time))


Average data wait time:  3757.3636363636365
Average forward + backward pass time:  84898.27272727272
Average move data to device time:  158.36363636363637


In [76]:
# print in milliseconds
print("Average data wait time in milliseconds: ", sum(data_wait_time)/len(data_wait_time)/1000)
print("Average forward + backward pass time: ", sum(forward_and_backward_pass_time)/len(forward_and_backward_pass_time)/1000)
print("Average move data to device time in milliseconds: ", sum(move_data_to_device_time)/len(move_data_to_device_time)/1000)


Average data wait time in milliseconds:  3.7573636363636367
Average forward + backward pass time:  84.89827272727273
Average move data to device time in milliseconds:  0.15836363636363637
